## <div class="alert alert-block alert-success"><b>**Business problem and plan for a solution**  </b></div>
The goal is to predict the probability of default of a loan. This can be used to automate approving and declinin loan applications.
**The hypothesis is to check:**
- what features  correlate the most with Loan status
- issuing higher grade loans to clients with better credit history and less loans to those with worse credit history
- how ownership of the property influances the default
- what age range is at risk
- if higher interest rates make risk to default on loans
    
 Will be perfomed:
- EDA
- Cleaning, feature engeeniring
- Assosiation and correlations of the variables
- Prediction using Models: Classification Logistics Regression

## <div class="alert alert-block alert-success"><b>**Dataset description**</div>
 N|Feature Name | Description |
--| ----------- | ----------- |
1 | person_age | Age
2 | person_income | Annual Income
3 | person_home_ownership | Home ownership
4 | person_emp_length | Employment length (in years)
5 | loan_intent | Loan intent
6 | loan_grade | Loan grade
7 | loan_amnt | Loan amount
8 | loan_int_rate | Interest rate
9 | loan_status | Loan status (0 is non default 1 is default)
10| loan_percent_income | Percent income
11| cb_person_default_on_file | Historical default
12| cb_preson_cred_hist_length | Credit history length
  

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')
import seaborn as sns
#import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as pyo

from sklearn.feature_selection import VarianceThreshold

import requests
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

In [3]:
df_original = pd.read_csv("credit_risk_dataset.csv.zip", skipinitialspace = True)
df_original.describe(percentiles=[0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,32581.000000,3.258100e+04,31686.000000,32581.000000,29465.000000,32581.000000,32581.000000,32581.000000
mean,27.734600,6.607485e+04,4.789686,9589.371106,11.011695,0.218164,0.170203,5.804211
std,6.348078,6.198312e+04,4.142630,6322.086646,3.240459,0.413006,0.106782,4.055001
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
0%,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
10%,22.000000,2.859000e+04,0.000000,3000.000000,6.910000,0.000000,0.050000,2.000000
20%,23.000000,3.500000e+04,1.000000,4400.000000,7.510000,0.000000,0.080000,3.000000
30%,24.000000,4.200000e+04,2.000000,5400.000000,8.900000,0.000000,0.100000,3.000000
40%,25.000000,4.900000e+04,3.000000,6750.000000,10.250000,0.000000,0.120000,4.000000
50%,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000


In [4]:
# Get test sample and remove test sample from train data
test = df_original.sample(frac = 0.1, random_state = 42).copy()
df = df_original.drop(test.index)

In [5]:
y_test = test['loan_status']
X_test = test.drop(columns = 'loan_status')

In [6]:
del test

In [7]:

df.describe(percentiles=[0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,29323.000000,2.932300e+04,28521.000000,29323.000000,26523.000000,29323.000000,29323.000000,29323.00000
mean,27.754288,6.617659e+04,4.783177,9599.156805,11.011271,0.216554,0.170218,5.81107
std,6.369895,6.318703e+04,4.092382,6330.565134,3.235866,0.411903,0.106869,4.05248
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.00000
0%,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.00000
10%,22.000000,2.880000e+04,0.000000,3000.000000,6.910000,0.000000,0.050000,2.00000
20%,23.000000,3.500000e+04,1.000000,4400.000000,7.510000,0.000000,0.080000,3.00000
30%,24.000000,4.200000e+04,2.000000,5465.000000,8.900000,0.000000,0.100000,3.00000
40%,25.000000,4.900000e+04,3.000000,6725.000000,10.250000,0.000000,0.120000,4.00000
50%,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.00000


In [10]:
df.shape, X_test.shape

((29323, 12), (3258, 11))

In [11]:
config = {
    'threshold_age': 50}

In [ ]:
def preproc_data(data, config):
    
    df_train = data.copy()
    df_train.loc[df_train['person_age']>config['threshold_age'], 'person_age'] = config['threshold_age']
    df_train.loc[df_train['person_income']>225200.0, 'person_income'] = 225200.0
    df_train.loc[df_train['person_emp_length']>18, 'person_emp_length'] = 18
    df_train.loc[df_train['person_home_ownership']=='OTHER', "person_home_ownership"] = "RENT"
    return df_train

In [ ]:
# df.loc[df['person_age']>50, 'person_age'] = 50
# df.loc[df['person_income']>225200.0, 'person_income'] = 225200.0
# df.loc[df['person_emp_length']>18, 'person_emp_length'] = 18

In [ ]:
df = preproc_data(data =df , config = config)

In [ ]:
for jcol in df.dtypes[df.dtypes=='object'].index:
    tmp_vc = df[jcol].value_counts(normalize=True).to_frame()
    tmp_vc.columns = ['share']
    mean_t = df.groupby(jcol)['loan_status'].mean().to_frame()
    mean_t.columns = ['mean_loan_status']
    print(jcol, '\n', tmp_vc.join(mean_t).round(4).sort_values('mean_loan_status'),'\n')

In [ ]:
df.groupby(['meta_grade', 'person_home_ownership']
)['loan_status'].mean().unstack(-1).style.background_gradient(cmap=cm)

In [ ]:
df.groupby(['loan_grade', 'person_home_ownership']
)['loan_status'].mean().unstack(-1).style.background_gradient(cmap=cm)

We will combine together similar groups that don't differ much

In [ ]:
# df.query('loan_grade == "G" & person_home_ownership == "MORTGAGE"')

In [ ]:
df['meta_grade'] = df['loan_grade']

In [ ]:
df.loc[(df['person_home_ownership'] == 'OWN') 
        & (df['loan_grade'].isin(['A', 'B', 'C', 'D'])), 'meta_grade'] = 'A'
df.loc[(df['person_home_ownership'] == 'RENT') 
        & (df['loan_grade'].isin(['D', 'E', 'F'])), 'meta_grade'] = 'D'
df.loc[(df['person_home_ownership'] == 'OWN') 
        & (df['loan_grade'].isin(['E', 'F'])), 'meta_grade'] = 'E'
df.loc[(df['person_home_ownership'] == 'RENT') 
        & (df['loan_grade'].isin(['B', 'C'])), 'meta_grade'] = 'B'

In [ ]:
df.groupby(['person_home_ownership', 'loan_intent']
)['loan_status'].mean().unstack(-1).style.background_gradient(cmap=cm)

In [ ]:
df.loc[(df['loan_intent'] == 'VENTURE') 
        & (df['person_home_ownership'].isin(['MORTGAGE', 'OWN'])), 'person_home_ownership'] = 'MORTGAGE'

In [ ]:
df.groupby(['loan_grade', 'loan_intent'])['loan_status'].mean().unstack(-1).style.background_gradient(cmap=cm)

In [ ]:
df.loc[(df['loan_intent'] == 'DEBTCONSOLIDATION') & (df['loan_grade'].isin(['D','E', 'F', 'G'])), 'loan_grade'] = 'D'
df.loc[(df['loan_intent'] == 'MEDICAL') & (df['loan_grade'].isin(['E', 'F', 'G'])), 'loan_grade'] = 'E'

In [ ]:
df.groupby(['loan_grade', 'loan_percent_income'])['loan_status'].mean().unstack(-1).style.background_gradient(cmap=cm)

In [ ]:
df.groupby(['loan_percent_income'])['loan_status'].mean().head(40)

In [ ]:
df['q_loan_percent_income'] = pd.qcut(df['loan_percent_income'],bins = 5)

In [ ]:
df['bin_loan_percent_income'] = pd.cut(df['loan_percent_income'],bins = 5, labels = ['1','2',' 3', '4', '5'])
df_cat = df.select_dtypes(include = ['object', 'category'])
df_cat

In [ ]:
from category_encoders import TargetEncoder
encoder = TargetEncoder()
df['encoded_bin_loan_percent_income'] = encoder.fit_transform(df['bin_loan_percent_income'], df['loan_status'])
df['encoded_bin_loan_percent_income']

In [ ]:
df.drop(columns = ['bin_loan_percent_income', 'loan_percent_income'], inplace = True)
df.columns

In [ ]:
null_values = (df.isnull().sum().sort_values(ascending = False)*100 / len(df)).round(2)
null_values

In [ ]:
# Fill Null values with mean as they are only in numerical columns and outliers are already removed
df.fillna(df.mean(), inplace = True)

In [ ]:
#df[['person_emp_length', 'loan_int_rate']].describe()

In [ ]:
# x = df.drop(labels = ['loan_status'], axis = 1)
# y = df.loan_status
#df['loan_status'] = df['loan_status'].astype('category', copy=False)
# df_num = df.select_dtypes(include = 'number')
# df_cat = df.select_dtypes(include = ['object', 'category'])
# df_num.shape, df_cat.shape

In [ ]:
# # summarize the number of rows with missing values for each column
# for i in range(df.shape[1]):
#     # count number of rows with missing values
#     n_miss = df.iloc[:,i].isnull().sum()
#     perc = n_miss / df.shape[0] * 100
#     print('> %d, Missing: %d (%.1f%%)' % (i, n_miss, perc))

In [ ]:
loan_st = df['loan_status'].value_counts(normalize = True)
ax = loan_st.plot(kind = 'bar')
ax.bar_label(ax.containers[-1], label_type='edge')

plt.title("Loan Status",fontsize=10)
plt.show()
print('Our target var is imbalanced')
print(df['loan_status'].value_counts())

**One of the treatment against imbalanced data is Resampling (Oversampling and Undersampling**

In [ ]:
# from sklearn.utils import resample
# #create two different dataframe of majority and minority class 
# df_majority = df[(df['loan_status']==0)] 
# df_minority = df[(df['loan_status']==1)] 
# # upsample minority class
# df_minority_upsampled = resample(df_minority, 
#                                  replace=True,    # sample with replacement
#                                  n_samples= 25473, # to match majority class
#                                  random_state=42)  # reproducible results
# # Combine majority class with upsampled minority class
# df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [ ]:
#df_upsampled.loan_status.value_counts()

In [ ]:
#sns.countplot(df_upsampled['loan_status'])

## <div class="alert alert-block alert-success"><b>EDA and Data Analysis</div>

In [ ]:
q = df[['loan_status', 'encoded_bin_loan_percent_income']]
f, ax = plt.subplots(figsize = (12, 6));
sns.boxplot(x = 'loan_status', y = 'encoded_bin_loan_percent_income', data = q, palette = 'Set1')
plt.title('Loan % income for Loan Status')
plt.xlabel('Loan Status', fontsize = 15)
plt.ylabel('Loan Percent Income', fontsize = 15)

In [ ]:
fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(26, 10))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.boxplot(y=k, x = 'loan_status', data=df, ax=axs[index],palette = 'Set1')
    index += 1
#plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

In [ ]:
df.corrwith(df['loan_status']).sort_values(axis=0, ascending = False) # by default it is pearson correlation

**We can pay attention to loan_percent_income and loan_int_rate as they correlate the most**

In [ ]:
# Make lists of categorical columns and numerical columns
#df['loan_status'] = df['loan_status'].astype('category', copy=False)
categ_col = df.select_dtypes(include = ['object', 'category']).columns.tolist()
num_col = df.select_dtypes(include='number').columns.tolist()

In [ ]:
#loan_st = df['person_home_ownership'].value_counts(normalize = True)
ax = df['person_home_ownership'].value_counts(normalize = True).plot(kind = 'bar', figsize =(15, 6))
ax.bar_label(ax.containers[-1], label_type='edge', fontsize=20)
plt.title("Proportion of Person Home Ownership",fontsize=18)
plt.show()

In [ ]:
plot_df = df.groupby(['person_home_ownership', 'loan_status']).size().unstack(-1)
plot_df = round(plot_df/plot_df.sum(), 4)#.sum()
#plot_df/plot_df.sum()
ax = plot_df.plot(kind = 'bar', figsize = (12, 6))
ax.bar_label(ax.containers[-1], label_type='edge', fontsize=14)
ax.bar_label(ax.containers[-2], label_type='edge', fontsize=14);

In [ ]:
plot_df = df.groupby(['loan_intent', 'loan_status']).size().unstack(-1)
plot_df = round(plot_df/plot_df.sum(), 4)#.sum()
#plot_df/plot_df.sum()
ax = plot_df.plot(kind = 'bar', figsize = (12, 6))
ax.bar_label(ax.containers[-1], label_type='edge', fontsize=14)
ax.bar_label(ax.containers[-2], label_type='edge', fontsize=14);

In [ ]:
#loan_st = df['loan_intent'].value_counts(normalize = True)
ax = df['loan_intent'].value_counts(normalize = True).plot(kind = 'bar', figsize =(12, 6))
ax.bar_label(ax.containers[-1], label_type='edge', fontsize=20)
plt.title("Propotion Of Loan Intent",fontsize=16)
plt.xticks(rotation = 0, fontsize=12)
plt.show()

In [ ]:
plot_df = df.groupby(['loan_intent', 'loan_status']).agg('size').unstack(-1)
plot_df = plot_df/plot_df.iloc[:].sum()
plot_df

In [ ]:
plot_df = df.groupby(['loan_intent', 'loan_status']).agg('size').unstack(-1)
plot_df = round(plot_df/plot_df.iloc[0:4].sum(), 4)#.sum()
#plot_df/plot_df.sum()
ax = plot_df.plot(kind = 'bar', figsize = (12, 6))
ax.bar_label(ax.containers[-1], label_type='edge', fontsize=14)
ax.bar_label(ax.containers[-2], label_type='edge', fontsize=14);

In [ ]:
ax = df['loan_grade'].value_counts(normalize = True).plot(kind = 'bar', figsize =(15, 6), color = '#d62728')
ax.bar_label(ax.containers[-1], label_type='edge', fontsize=20)
plt.title("Propotion Of Loan Grade",fontsize=16); plt.xticks(rotation = 0, fontsize=12); plt.show()

In [ ]:
loan_st = df['cb_person_default_on_file'].value_counts(normalize = True)
ax = loan_st.plot(kind = 'bar', figsize =(12, 5), color = '#1f77b4')
ax.bar_label(ax.containers[-1], label_type='edge', fontsize=16)
plt.title("Propotion Of CB Person Default On File",fontsize=12); plt.xticks(rotation = 0, fontsize=12); plt.show()

In [ ]:
# cat_col = ['person_home_ownership',
#   'loan_intent',
#   'loan_grade',
#   'cb_person_default_on_file']
# df[cat_col].mode()

In [ ]:
fig, axes = plt.subplots(4,2, figsize = (34, 34))
splot = sns.countplot('person_home_ownership', data = df, ax = axes[0,0], palette = 'Set1')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),  ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot = sns.countplot('person_home_ownership', hue = 'loan_status', data=df, ax = axes[0,1], palette = 'Dark2')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),  ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot = sns.countplot('loan_intent', data = df, ax = axes[1,0], palette = 'Set1')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),  ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot = sns.countplot('loan_intent', hue = 'loan_status', data=df, ax = axes[1,1], palette = 'Dark2')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),  ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot = sns.countplot('loan_grade', data = df, ax = axes[2,0], palette = 'Set1')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),  ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot = sns.countplot('loan_grade', hue = 'loan_status', data=df, ax = axes[2,1], palette = 'Dark2')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),  ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot = sns.countplot('cb_person_default_on_file', data = df, ax = axes[3,0], palette = 'Set1')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),  ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

splot = sns.countplot('cb_person_default_on_file', hue = 'loan_status', data=df, ax = axes[3,1], palette = 'Dark2')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()),  ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')


## <div class="alert alert-block alert-success"><b>Statistics</div>

In [ ]:
df_num = df.select_dtypes(include = 'number')
df_cat = df.select_dtypes(include = ['object', 'category'])
df_num.shape, df_cat.shape

In [ ]:
df_num.columns; df_cat.columns;

In [ ]:
# #df['loan_status'] = df['loan_status'].astype('int32')
# plt.figure(figsize = (16, 8))
# plt.title('Credit Risk Correlation plot (Pearson)')
# corr = df.corr()
# sns.heatmap(corr, xticklabels = corr.columns, yticklabels = corr.columns, 
#             annot = True, mask = np.triu(np.ones_like(df.corr(), dtype = bool)), 
#             linewidths = .1, cmap = sns.diverging_palette(230, 20))
# plt.show()

In [ ]:
#mask = np.triu(np.ones_like(df.corr(), dtype = bool))
plt.figure(figsize = (16, 8))
plt.title('Credit Risk Correlation plot (Pearson)')
sns.heatmap(df.corr(), annot = True, mask = np.triu(np.ones_like(df.corr(), dtype = bool)), cmap = sns.diverging_palette(230, 20));

**Correlations between features are not high**

In [ ]:
cor = df.corr()
cor[((cor >= .5) | (cor<= -.5))]

In [ ]:
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in df_num.items():
    sns.distplot(v, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
# Visualizing the distribution of observations in a dataset (analogous to a histogram.) 
# KDE represents the data using a continuous probability density curve 
fig, axes = plt.subplots(4,2, figsize = (24, 14)); axes = axes.ravel() 
cols =df_num.columns[:]
for col, ax in zip(cols, axes):
    data = df_num
    sns.kdeplot(data=data, x=col, shade=True, ax = ax, palette = 'Set1')
    ax.set(title=f'Distribution of Variable: {col}', xlabel=None)
#fig.delaxes(axes[8])
ax.xaxis.label.set_size(20); fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0); plt.show()

In [ ]:
# Visualizing categorical variables with Loan status color  encoding
fig, axes = plt.subplots(2,2, figsize = (24, 14))
axes = axes.ravel() 
cols =df_cat.columns[:]
for col, ax in zip(cols, axes):
    data = df
    sns.histplot(data=data, x=col, ax = ax, bins=20, hue= 'loan_status',palette = 'Set1')
    ax.set(title=f'Distribution of Variable: {col}', xlabel=None)

#fig.delaxes(axes[8])
ax.xaxis.label.set_size(20)
fig.tight_layout()
plt.show()

In [ ]:
df_num.skew()

In [ ]:
fig, axes = plt.subplots(8, 1, figsize = (20, 14))
axes = axes.ravel() 
cols =df.select_dtypes(include = [np.number]).columns[:]

for col, ax in zip(cols, axes):
    data = df.select_dtypes(include = [np.number])
    sns.boxplot(data=data, x=col, ax = ax, palette = 'Set1')
    ax.set(title=f'Distribution of Variable: {col}', xlabel=None)
#fig.delaxes(axes[8])
fig.tight_layout()
plt.show()

In [ ]:
# def remove_outlier(df_in, col_name):
#     q1 = df_in[col_name].quantile(0.25)
#     q3 = df_in[col_name].quantile(0.75)
#     iqr = q3-q1 #Interquartile range
#     fence_low  = q1-1.5*iqr
#     fence_high = q3+1.5*iqr
#     df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
#     return df_out

In [ ]:
# # Treating outliers
# def outlier_detect(df):
#     for i in df.describe().columns:
#         Q1=df.describe().at['25%',i]
#         Q3=df.describe().at['75%',i]
#         IQR=Q3 - Q1
#         LTV=Q1 - 1.5 * IQR
#         UTV=Q3 + 1.5 * IQR
#         x=np.array(df[i])
#         p=[]
#         for j in x:
#             if j < LTV or j>UTV:
#                 p.append(df[i].median())
#             else:
#                 p.append(j)
#         df[i]=p
#     return df

In [ ]:
# df_outliers = df_num[['person_age',
#   'person_income',
#   'person_emp_length',
#   'loan_amnt',
#   'loan_int_rate',
  
#   'loan_percent_income',
#   'cb_person_cred_hist_length']]

In [ ]:
# df_outlier_clean = outlier_detect(df_outliers)
# df_outlier_clean.head(2)

In [ ]:
# fig, axes = plt.subplots(8, 1, figsize = (34, 54))
# axes = axes.ravel() 
# cols =df_outlier_clean.columns[:]

# for col, ax in zip(cols, axes):
#     data = df_outlier_clean
#     sns.boxplot(data=data, x=col, ax = ax, palette = 'Set1')
#     ax.set(title=f'Distribution of Variable: {col}', xlabel=None)
# #fig.delaxes(axes[8])
# fig.tight_layout()
# plt.show()

In [ ]:
#df_outlier_clean.head()

In [ ]:
# Pairplot with outliers
#df['loan_status'] = df['loan_status'].astype()
sns.pairplot(df,hue="loan_status")

In [ ]:
# df_clean = df_outlier_clean.join(df_cat)
# df_clean.shape

In [ ]:
# df_all = df_clean.join(df_num['loan_status'])
# df_all.isnull().sum()

In [ ]:
#df_all = df_all.fillna(df_all.median())

In [ ]:
# Pairplot without outliers
#sns.pairplot(df_all, hue="loan_status")

In [ ]:
# cross_loan_income = pd.crosstab(index = df_all['bin_loan_percent_income'], columns = df_all['loan_percent_income'], normalize = 'columns').round(4)*100
# cross_loan_income

In [ ]:
# w = df_all['bin_loan_percent_income'].value_counts()
# w

In [ ]:
# cross_index = pd.crosstab(index = df_all['loan_intent'], columns = df_all['loan_grade'], normalize='index').round(4)*100
# cross_index

In [ ]:
# cross_df = pd.crosstab(index = df_all['loan_intent'], columns = df_all['loan_grade'], margins=True, margins_name='Total', normalize = True).round(4) * 100
# cross_df

In [ ]:
# cross_col = pd.crosstab(index = df_all['loan_intent'], columns = df_all['loan_status'], normalize = 'index').round(4)*100
# cross_col

In [ ]:
#df_num.loan_status.value_counts()

In [ ]:
# cross_hist_length = pd.crosstab(index = df_all['cb_person_cred_hist_length'], columns = df_all['loan_status'], normalize ='index').round(4)*100
# cross_hist_length.T

In [ ]:
# cross_hist_length.plot(kind = 'line')
# cross_hist_length.plot(kind = 'area')


In [ ]:
# corr_matrix = df.corr(method = 'pearson')
# corr_matrix.style.background_gradient(cmap = 'coolwarm')

In [ ]:
plt.figure(figsize=(10, 5))
plt.xticks(rotation=75)
plt.title('Age by Ownership')
sns.barplot(x=df_all['person_home_ownership'], y = df_all['person_age'])

In [ ]:
plt.figure(figsize=(10, 5))
plt.xticks(rotation=75)
plt.title('Relationship between Income and Loan Grade' )
sns.barplot(x=df_all['person_income'], y = df_all['loan_grade'])

In [ ]:
# make long feature names shorter
df_ml= df_all.rename(columns = {"cb_person_default_on_file":"default_hist", "cb_person_cred_hist_length": "cr_hist_len"})

In [ ]:
# percentage of null values from loan int rate col
df_ml.loan_int_rate.isnull().sum() / df_ml.shape[0]

In [ ]:
# df_ml['person_emp_length'].fillna((df_ml['person_emp_length'].median()), inplace=True)
# df_ml['loan_int_rate'].fillna((df_ml['loan_int_rate'].median()), inplace = True)

In [ ]:
num_col = df_ml.select_dtypes(include = 'number')
char_col = df_ml.select_dtypes(include = ['object', 'category'])

In [ ]:
num_col.drop('loan_percent_income', axis = 1, inplace = True)

In [ ]:
num_col

In [ ]:
encoded_char_col = pd.get_dummies(char_col)

In [ ]:
cr_clean2 = pd.concat([num_col, encoded_char_col], axis=1)
cr_clean2.tail(2)

In [ ]:
# cr_clean3 = num_col.join(encoded_char_col)
# cr_clean3

## <div class="alert alert-block alert-success"><b>Model</div>

 When normalizing the features, it is important to store the values used for normalization - the mean value and the standard deviation used for the computations. After learning the parameters from the model, we often want to predict the prices of houses we have not seen before. Given a new x value (living room area and number of bed- rooms), we must first normalize x using the mean and standard deviation that we had previously computed from the training set.

In [ ]:
from sklearn import model_selection,linear_model, metrics
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

In [ ]:
df_cat.columns

In [ ]:
df_cat = pd.get_dummies(df.select_dtypes(include = ['object', 'category']))
df = pd.concat([df_num, df_cat], axis=1)

In [ ]:
# Split Train and Test Sets
# Y = cr_clean2['loan_status']
# X = cr_clean2.drop('loan_status',axis=1)
Y = df['loan_status']
X = df.drop(['loan_status'], axis = 1)
#one_hot_encoded_training_predictors = pd.get_dummies
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, random_state=2020, test_size=.32)
log_clf = linear_model.LogisticRegression()
log_clf.fit(x_train, np.ravel(y_train))

In [ ]:
Y = test
X = df.drop(['loan_status'], axis = 1)
#one_hot_encoded_training_predictors = pd.get_dummies
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, random_state=2020, test_size=.32)
log_clf = linear_model.LogisticRegression()
log_clf.fit(x_train, np.ravel(y_train))

In [ ]:
y_pred = log_clf.predict(x_test)
score =accuracy_score(y_test,y_pred)
score

In [ ]:
pca_accuracy = round(log_clf.score(x_test,y_test),6)
pca_accuracy

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
confusion_matrix

**Let's display probabilities of default**

In [ ]:
prob_default = pd.DataFrame(log_clf.predict_proba(x_test)[:, 1], columns = ['prob_default'])
prob_default.head(2)

In [ ]:
prediction_default = pd.concat([y_test.reset_index(drop=True), prob_default], axis = 1)
prediction_default

In [ ]:
result = x_test.join(y_test)
result.iloc[:,3].shape

In [ ]:
col_effect = pd.DataFrame()
col_effect['col_names'] = X.columns
col_effect['col_coef'] = log_clf.coef_[0]
col_effect.sort_values(by='col_coef', ascending=False)

**If a coefficient is more positive, the person is more likely to default on the loan, when the coefficient is more negative the person is less likely to default.**

In [ ]:
# pd.DataFrame(log_clf.coef_[0],
#             X.columns,
#             columns = ['coef']).sort_values(by='coef', ascending=False)

In [ ]:
int_val = float(log_clf.intercept_)
print('The overall probablity of non default is {:.3%}'.format(int_val))

In [ ]:
int_val

In [ ]:
# scaler = MinMaxScaler(feature_range=[0, 1])
# data_rescaled = scaler.fit_transform(num_col)

# #Fitting the PCA algorithm with our Data
# pca = PCA().fit(data_rescaled)
# #Plotting the Cumulative Summation of the Explained Variance
# plt.figure()
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('Number of Components')
# plt.ylabel('Variance (%)') #for each component
# plt.title('Hotel Booking Dataset Explained Variance')
# plt.show()

In [ ]:
# random forest paraeters tuned using optuna. Code down below
random_forest_parameters = {'n_estimators': 102, 'max_depth': 5, 'min_samples_split': 5, 'min_samples_leaf': 3}


# choosing model
model_name = RandomForestClassifier(n_jobs=-1, random_state=random_seed, **random_forest_parameters)

# giving all values to pipeline
rf_pipe = make_pipeline(preprocessor,model_name)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, random_state=2020, test_size=.30)
#Start of gradient boosted tree
xgb_model = xgb.XGBClassifier() # initialize tree

xgb_model.fit(x_train, np.ravel(y_train)) # train tree

predict_xgb = xgb_model.predict_proba(x_test) # 1st col = pred val, 2nd col = pred prob
predict_xgb_prob = pd.DataFrame(predict_xgb[:,1],columns = ['Default Probability'])
pd.concat([predict_xgb_prob, y_test.reset_index(drop=True)],axis=1)

In [ ]:
pca_accuracy = round(xgb_model.score(x_test,y_test),6)
pca_accuracy